In [1]:
# Activate intractive functionality of matplotlib
%matplotlib ipympl
# Activate auto reload 
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
# import libraries
import os
import functools
import numpy as np
from scipy.optimize import curve_fit
from ipywidgets import fixed, interact_manual, widgets
from ipywidgets import HBox, VBox
from IPython.display import clear_output
from IPython.display import display
from ipywidgets import Output
import warnings
# Ignore all warnings
warnings.filterwarnings("ignore")

# Local module and scripts
from pyccapt.calibration.calibration_tools import tools, share_variables, calibration
from pyccapt.calibration.calibration_tools import widgets as wd
from pyccapt.calibration.data_tools import data_tools, data_loadcrop, dataset_path_qt

In [2]:
button = widgets.Button(
    description='load dataset',
)
@button.on_click
def open_file_on_click(b):
    """
    Event handler for button click event.
    Prompts the user to select a dataset file and stores the selected file path in the global variable dataset_path.
    """
    global dataset_path
    dataset_path = dataset_path_qt.gui_fname().decode('ASCII')
button

Button(description='load dataset', style=ButtonStyle())

In [4]:
tdc, pulse_mode, flightPathLength_d, t0_d, max_mc, det_diam = wd.dataset_instrument_specification_selection()
display(tdc, det_diam, pulse_mode, flightPathLength_d)

Dropdown(description='Data mode:', options=('PyCCAPT', 'leap_epos', 'leap_pos', 'ato_v6'), value='PyCCAPT')

FloatText(value=80.0, description='Detector diameter:')

Dropdown(description='Pulse mode:', options=('voltage', 'laser'), value='voltage')

FloatText(value=110.0, description='Flight path length:')

$$\textbf{You can specify which dataset to use in below block}$$

In [7]:
# exctract needed data from Pandas data frame as an numpy array
# create an instance of the Variables opject
variables = share_variables.Variables()
variables.pulse_mode = pulse_mode
dataset_main_path = os.path.dirname(dataset_path)
dataset_main_path = os.path.dirname(dataset_main_path)
dataset_name_with_extention = os.path.basename(dataset_path)
variables.dataset_name = os.path.splitext(dataset_name_with_extention)[0]
variables.result_data_path = dataset_main_path + '/mc_calibration/'
variables.result_data_name = variables.dataset_name
variables.result_path = dataset_main_path + '/' + '/mc_calibration/'

if not os.path.isdir(variables.result_path):
    os.makedirs(variables.result_path, mode=0o777, exist_ok=True)
    
# Create data farame out of hdf5 file dataset
data = data_tools.load_data(dataset_path, tdc.value, mode='processed')
# extract data from the path and create the Variable object
data_tools.extract_data(data, variables, flightPathLength_d.value, max_mc.value)

The maximum time of flight: 5010


In [8]:
print('The data will be saved on the path:', variables.result_data_path)
print('=============================')
print('The dataset name after saving is:', variables.result_data_name)
print('=============================')
print('The figures will be saved on the path:', variables.result_path)
print('=============================')
print('Total number of Ions:', len(data))
data

The data will be saved on the path: D:/pyccapt/tests/data/Laura/1684_Sep-29-2023_13-35_SS430_B_20_hp_test1/mc_calibration/
The dataset name after saving is: 1684_Sep-29-2023_13-35_SS430_B_20_hp_test1
The figures will be saved on the path: D:/pyccapt/tests/data/Laura/1684_Sep-29-2023_13-35_SS430_B_20_hp_test1//mc_calibration/
Total number of Ions: 1220925


,x (nm),y (nm),z (nm),mc_c (Da),mc (Da),high_voltage (V),pulse,start_counter,t_c (ns),t (ns),x_det (cm),y_det (cm),pulse_pi,ion_pp
0,0.0,0.0,0.0,0.0,70.627667,3922.669922,588.400513,6927,0.0,1089.352173,3.226122,0.561633,0,0
1,0.0,0.0,0.0,0.0,32.017647,3922.669922,588.400513,7960,0.0,740.828613,-2.628572,1.204898,1033,2
2,0.0,0.0,0.0,0.0,30.097386,3922.669922,588.400513,8275,0.0,707.361572,0.460408,1.880816,315,1
3,0.0,0.0,0.0,0.0,31.301152,3922.669922,588.400513,8292,0.0,713.341736,0.476735,0.943673,17,1
4,0.0,0.0,0.0,0.0,31.665813,3922.669922,588.400513,8383,0.0,724.005920,-1.808980,-0.551837,91,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1220920,0.0,0.0,0.0,0.0,30.766599,4824.825195,723.723755,15804,0.0,658.086792,2.755918,-0.313469,548,1
1220921,0.0,0.0,0.0,0.0,30.640980,4824.825195,723.723755,15890,0.0,653.485107,1.949388,1.580408,86,1
1220922,0.0,0.0,0.0,0.0,29.433193,4824.825195,723.723755,16216,0.0,636.936768,-1.146122,-1.769796,326,1
1220923,0.0,0.0,0.0,0.0,31.071022,4824.825195,723.723755,16218,0.0,657.956543,-0.914286,-2.354286,2,1


In [9]:
interact_manual(data_loadcrop.plot_crop_experiment_history, data=fixed(data), variables=fixed(variables), max_tof=widgets.FloatText(value=variables.max_tof), frac=widgets.FloatText(value=1.0),
                bins=fixed((1200,800)), figure_size=fixed((7,3)),
               draw_rect=fixed(False), data_crop=fixed(False), pulse_plot=widgets.Dropdown(options=[('False', False), ('True', True)]), dc_plot=widgets.Dropdown(options=[('True', True), ('False', False)]),
               pulse_mode=widgets.Dropdown(options=[('voltage', 'voltage'), ('laser', 'laser')]), save=widgets.Dropdown(options=[('True', True), ('False', False)]),
               figname=widgets.Text(value='exp_hist'));

interactive(children=(FloatText(value=5010.0, description='max_tof'), FloatText(value=1.0, description='frac')…

In [10]:
interact_manual(tools.mc_hist_plot, variables=fixed(variables), bin_size=widgets.FloatText(value=0.1), 
                mode=widgets.Dropdown(options=[('mc', 'mc'), ('tof', 'tof')]), prominence=widgets.IntText(value=50), distance=widgets.IntText(value=100), 
                lim=widgets.IntText(value=5000), percent=widgets.IntText(value=50), selector=fixed('None'), plot=fixed(True), figname=widgets.Text(value='hist'),
                peaks_find_plot=fixed(False));

interactive(children=(FloatText(value=0.1, description='bin_size'), Dropdown(description='mode', options=(('mc…

$$\textbf{Below plotted graph offers peak selection. You can select relevant peaks you want for computation.}$$

In [11]:
bin_size=widgets.FloatText(value=0.1, description='bin size:')
prominence=widgets.IntText(value=100, description='peak prominance:')
distance=widgets.IntText(value=500, description='peak distance:')
lim_tof=widgets.IntText(value=400, description='lim tof/mc:')
percent=widgets.IntText(value=50, description='percent MRP:')
index_fig = widgets.IntText(value=1, description='fig index:')
plot_peak = widgets.Dropdown(
    options=[('False', False), ('True', True)],
    description='plot peak:'
)

def hist_plot(variables, plot):
    
    with out:
        clear_output(True)
    
        bin_size_p = bin_size.value
        prominence_p = prominence.value
        distance_p = distance.value
        lim_tof_p = lim_tof.value
        percent_p = percent.value
        index_fig_p = index_fig.value
        plot_peak_p = plot_peak.value
        tools.mc_hist_plot(variables, bin_size_p, mode='mc', prominence=prominence_p, distance=distance_p, percent=percent_p, selector='rect', plot=True, figname=index_fig_p, lim=lim_tof_p,
                          peaks_find_plot=plot_peak_p)

In [12]:
sample_size_b = widgets.IntText(value=11, description='sample size:') 
index_fig_b = widgets.IntText(value=1, description='fig index:')
maximum_cal_method_b = widgets.Dropdown(
    options=[('mean', 'mean'), ('histogram', 'histogram')],
    description='calib method:'
)
plot_b = widgets.Dropdown(
    options=[('False', False), ('True', True)],
    description='plot fig:'
)

save_b = widgets.Dropdown(
    options=[('False', False), ('True', True)],
    description='save fig:'
)
apply_b = widgets.Dropdown(
    options=[('all', 'all'), ('temporal', 'temporal'),],
    description='apply mode:'
)
def bowl_correction(dld_x, dld_y, dld_highVoltage, variables):
    with out:
        sample_size_p = sample_size_b.value
        index_fig_p = index_fig_b.value
        plot_p = plot_b.value
        save_p = save_b.value
        maximum_cal_method_p = maximum_cal_method_b.value
        calibration.bowl_correction_main(dld_x, dld_y, dld_highVoltage, variables, det_diam.value, sample_size=sample_size_p, maximum_cal_method=maximum_cal_method_p, apply_local=apply_b.value,
                                         calibration_mode='mc', index_fig=index_fig_p, plot=plot_p, save=save_p)


In [13]:
sample_size_v =widgets.IntText(value=100, description='sample size:') 
index_fig_v = widgets.IntText(value=1, description='fig index:')
plot_v = widgets.Dropdown(
    options=[('False', False), ('True', True)],
    description='plot fig:'
)
save_v = widgets.Dropdown(
    options=[('False', False), ('True', True)],
    description='save fig:'
)
mode_v = widgets.Dropdown(
    options=[('ion_seq', 'ion_seq'), ('voltage', 'voltage')],
    description='sample mode:'
)
peak_mode = widgets.Dropdown(
    options=[('peak', 'peak'), ('mean', 'mean'), ('median', 'median')],
    description='peak mode:'
)
num_cluster = widgets.IntText(value=1, description='num_cluster:')
apply_v = widgets.Dropdown(
    options=[('all', 'all'), ('voltage', 'voltage'), ('voltage_temporal', 'voltage_temporal')],
    description='apply mode:'
)
def vol_correction(dld_highVoltage, variables):
    with out:
        sample_size_p = sample_size_v.value
        index_fig_p = index_fig_v.value
        plot_p = plot_v.value
        save_p = save_v.value
        mode_p = mode_v.value
        peak_mode_p = peak_mode.value
        calibration.voltage_corr_main(dld_highVoltage, variables, sample_size=sample_size_p, calibration_mode='mc',
                                        index_fig=index_fig_p, plot=plot_p, save=save_p, apply_local=apply_v.value, num_cluster=num_cluster.value, mode=mode_p, peak_mode=peak_mode_p)


In [14]:
pb_bowl = widgets.HTML(
    value=" ",
    placeholder='Status:',
    description='Status:',
)
pb_vol = widgets.HTML(
    value=" ",
    placeholder='Status:',
    description='Status:',
)
plot_button = widgets.Button(
    description='plot hist',
)
plot_stat_button = widgets.Button(
    description='plot stat',
)
reset_back_button = widgets.Button(
    description='reset back correction',
)
reset_button = widgets.Button(
    description='reset',
)
save_button = widgets.Button(
    description='save correction',
)
bowl_button = widgets.Button(
    description='bowl correction',
)
vol_button = widgets.Button(
    description='voltage correction',
)
bin_fdm = widgets.IntText(value=256, description='bin FDM:')
    
@plot_button.on_click
def plot_on_click(b, variables=variables, plot=True):
    hist_plot(variables, plot)

@plot_stat_button.on_click
def plot_stat_on_click(b, variables=variables, save=True):
    with out2:
        clear_output(True)
    with out:
        clear_output(True)
        calibration.plot_selected_statistic(variables, bin_fdm.value, index_fig.value, calibration_mode='mc', save=True)
    
@reset_back_button.on_click
def reset_back_on_click(b, variables=variables):
    variables.mc_calib = np.copy(variables.mc_calib_backup)
@reset_button.on_click
def reset_on_click(b, data=data, variables=variables, flightPathLength_d=flightPathLength_d.value, max_mc=max_mc.value):
    data_tools.extract_data(data, variables, flightPathLength_d, max_mc)
    
@save_button.on_click
def save_on_click(b, variables=variables):
    variables.mc_calib_backup = np.copy(variables.mc_calib)

@vol_button.on_click
def vol_on_click(b, dld_highVoltage=variables.dld_high_voltage, variables=variables):
    with out2:
        clear_output(True)
        pb_vol.value = "<b>Starting...</b>"
        if variables.selected_x1 == 0 or variables.selected_x2 == 0:
            print('Please first select a peak')
        else:
            print('Selected mc ranges are: (%s, %s)' %(variables.selected_x1, variables.selected_x2))
            vol_correction(dld_highVoltage, variables)
        pb_vol.value = "<b>Flished</b>"
@bowl_button.on_click
def bowl_on_click(b, dld_x_det=variables.dld_x_det, dld_y_det=variables.dld_y_det, dld_highVoltage=variables.dld_high_voltage, variables=variables):
    with out2:
        clear_output(True)
        pb_bowl.value = "<b>Starting...</b>"
        if variables.selected_x1 == 0 or variables.selected_x2 == 0:
            print('Please first select a peak')
        else:
            print('Selected mc ranges are: (%s, %s)' %(variables.selected_x1, variables.selected_x2))
            bowl_correction(dld_x_det, dld_y_det, dld_highVoltage, variables)  
        pb_bowl.value = "<b>Flished</b>"


tab1 = VBox(children=[bin_size, index_fig, prominence, distance, lim_tof, percent, bin_fdm, plot_peak])
tab2 = VBox(children=[VBox(children=[sample_size_b, index_fig_b, maximum_cal_method_b, apply_b, plot_b, save_b]), bowl_button, pb_bowl])
tab3 = VBox(children=[VBox(children=[sample_size_v, index_fig_v, mode_v, apply_v, num_cluster, peak_mode, plot_v, save_v]), vol_button, pb_vol])

tab = widgets.Tab(children=[tab1, tab2, tab3])
tab.set_title(0, 'tof/mc plot')
tab.set_title(1, 'bowl correction')
tab.set_title(2, 'voltage correction')



display(VBox(children=[tab,HBox(children=[plot_button, plot_stat_button, save_button, reset_back_button, reset_button])]))
out = Output()
out2 = Output()
display(out)

Output()

In [15]:
variables.mc_calib_backup = variables.mc_calib

In [16]:
variables.mc_calib = variables.mc_calib_backup

In [17]:
interact_manual(tools.mc_hist_plot, variables=fixed(variables), bin_size=widgets.FloatText(value=0.1), 
                mode=widgets.Dropdown(options=[('mc', 'mc')]), prominence=widgets.IntText(value=50), distance=widgets.IntText(value=100), 
                lim=widgets.IntText(value=400), percent=widgets.IntText(value=50), selector=fixed('peak'), plot=fixed(True), figname=widgets.Text(value='mc_hist_calibrated'),
               peaks_find_plot=widgets.Dropdown(options=[('True', True), ('False', False)]));


interactive(children=(FloatText(value=0.1, description='bin_size'), Dropdown(description='mode', options=(('mc…

In [18]:
isotopeTableFile = '../../../files/isotopeTable.h5'
dataframe = data_tools.read_hdf5_through_pandas(isotopeTableFile)
elementsList = dataframe['element']
elementIsotopeList = dataframe['isotope']
elementMassList =  dataframe['weight']
abundanceList = dataframe['abundance']

elements = list(zip(elementsList, elementIsotopeList, elementMassList, abundanceList))
dropdownList = []
for element in elements:
    tupleElement = ("{} ({}) ({:.2f})".format(element[0], element[1], element[3]), "{}({})[{}]".format(element[0], element[1], element[2]))
    dropdownList.append(tupleElement)

chargeList = [(1,1,),(2,2,),(3,3,),(4,4,)]
dropdown = wd.dropdownWidget(dropdownList,"Elements")
dropdown.observe(wd.on_change)


chargeDropdown = wd.dropdownWidget(chargeList,"Charge")
chargeDropdown.observe(wd.on_change_charge)

wd.compute_element_isotope_values_according_to_selected_charge()

buttonAdd = wd.buttonWidget("ADD")
buttonDelete = wd.buttonWidget("DELETE")
buttonReset = wd.buttonWidget("RESET")


display(dropdown, chargeDropdown, buttonAdd, buttonDelete, buttonReset)

def buttonAdd_f(b, variables):
    with out:
        clear_output(True)
        wd.onClickAdd(b, variables)
        display()
def buttonDelete_f(b, variables):
    with out:
        clear_output(True)
        wd.onClickDelete(b, variables)
        display()
def buttonResett_f(b, variables):
    with out:
        clear_output(True)
        wd.onClickReset(b, variables)
        display()

listMaterial = buttonAdd.on_click(functools.partial(buttonAdd_f, variables=variables))
buttonDelete.on_click(functools.partial(buttonDelete_f, variables=variables))
buttonReset.on_click(functools.partial(buttonResett_f, variables=variables))
# listMaterial = buttonAdd.on_click(wd.onClickAdd)
# buttonDelete.on_click(wd.onClickDelete)
# buttonReset.on_click(wd.onClickReset)
out = Output()
display(out)

Dropdown(description='Elements', options=(('H (1) (99.98)', 'H(1)[1.01]'), ('H (2) (0.01)', 'H(2)[2.01]'), ('H…

Dropdown(description='Charge', options=((1, 1), (2, 2), (3, 3), (4, 4)), value=1)

Button(description='ADD', icon='check', style=ButtonStyle(), tooltip='ADD')

Button(description='DELETE', icon='check', style=ButtonStyle(), tooltip='DELETE')

Button(description='RESET', icon='check', style=ButtonStyle(), tooltip='RESET')

Output()

In [19]:
peaks_chos = []
for i in range(len(variables.peaks_idx)):
    peaks_chos.append(variables.peak[variables.peaks_idx[i]])
peaks_chos = np.array(peaks_chos)
listMaterial = np.array(variables.listMaterial)
print('highest peak in the mc histogram:', peaks_chos)
print('highest peak in the ideal mc histogram:', listMaterial)

highest peak in the mc histogram: []
highest peak in the ideal mc histogram: []


In [17]:
# def shift(mc, a, c):
#     return  a * mc + c

def shift(mc, a, b, c):
    return  a * mc**b + c

# def shift(mc, a, b, c):
#     return a * ((mc - b)**2) + c

def shift_calib(mc, mc_ideal):
    
    fitresult, _ = curve_fit(shift, mc, mc_ideal, maxfev=2000)
    return fitresult

fitresult = shift_calib(peaks_chos,  variables.listMaterial)
variables.mc_calib = shift(variables.mc_calib_backup, *fitresult)

# from scipy import interpolate
# f_shift = interpolate.interp1d(np.array(variables.listMaterial).T, peaks_chos.T, fill_value="extrapolate")
# variables.mc_calib = f_shift(variables.mc_calib)



In [18]:
interact_manual(tools.mc_hist_plot, variables=fixed(variables), bin_size=widgets.FloatText(value=0.1), 
                mode=widgets.Dropdown(options=[('mc', 'mc')]), prominence=widgets.IntText(value=50), distance=widgets.IntText(value=100), 
                lim=widgets.IntText(value=400), percent=widgets.IntText(value=50), selector=fixed('peak'), plot=fixed(True), figname=widgets.Text(value='mc_hist_calibrated'),
               peaks_find_plot=fixed(True));


interactive(children=(FloatText(value=0.1, description='bin_size'), Dropdown(description='mode', options=(('mc…

In [19]:
data['mc_c (Da)'] = variables.mc_calib
data

,x (nm),y (nm),z (nm),mc_c (Da),mc (Da),high_voltage (V),pulse,start_counter,t_c (ns),t (ns),x_det (cm),y_det (cm),pulse_pi,ion_pp
0,0.0,0.0,0.0,26.999754,28.556046,5018.839844,1003.768005,6860,0.0,617.933228,1.080816,-2.804898,0,0
1,0.0,0.0,0.0,27.403896,30.019277,5018.839844,1003.768005,6949,0.0,618.173279,-1.609796,0.457143,89,2
2,0.0,0.0,0.0,113.503393,126.272476,5018.839844,1003.768005,7911,0.0,1253.614990,-1.260408,2.749388,962,1
3,0.0,0.0,0.0,27.022691,29.652876,5018.839844,1003.768005,8223,0.0,616.177612,-1.642449,0.875102,312,1
4,0.0,0.0,0.0,27.067992,29.081018,5018.839844,1003.768005,8328,0.0,620.703857,1.521633,2.367347,105,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10657311,0.0,0.0,0.0,26.701804,28.812703,6350.299805,1270.060059,4967,0.0,538.353027,0.238367,-0.244898,15,2
10657312,0.0,0.0,0.0,14.057703,15.092135,6350.299805,1270.060059,4987,0.0,404.381958,-0.777143,-1.283265,20,1
10657313,0.0,0.0,0.0,27.075017,29.215558,6350.299805,1270.060059,5041,0.0,546.850037,-0.698776,-1.443265,54,1
10657314,0.0,0.0,0.0,27.203200,28.946840,6350.299805,1270.060059,5094,0.0,555.161987,2.413061,1.426939,53,1


In [20]:
data_c = data.copy(deep = True)

In [21]:
# Remove negative mc
threshold = 0
mc_t = data_c['mc_c (Da)'].to_numpy()
mc_t_mask = (mc_t <= threshold)
print('The number of ions with negative mc are:', len(mc_t_mask[mc_t_mask==True]))
data_c.drop(np.where(mc_t_mask)[0], inplace=True)
data_c.reset_index(inplace=True, drop=True)

The number of ions with negative mc are: 0


In [22]:
interact_manual(data_tools.save_data, data=fixed(data_c), variables=fixed(variables),
                hdf=widgets.Dropdown(options=[('True', True), ('False', False)]),
                epos=widgets.Dropdown(options=[('False', False), ('True', True)]), 
                pos=widgets.Dropdown(options=[('False', False), ('True', True)]), 
                ato_6v=widgets.Dropdown(options=[('False', False), ('True', True)]), 
                csv=widgets.Dropdown(options=[('False', False), ('True', True)]));

interactive(children=(Dropdown(description='hdf', options=(('True', True), ('False', False)), value=True), Dro…